In [1]:
import torch
import pandas as pd
import pyaging as pya
import os

In [2]:
os.system("git clone https://github.com/kerepesi/MouseAgingClocks.git")

0

In [3]:
# This is supplementary table 1 in the paper, but it's in a Word document. Let's just use the .txt file from the GitHub above
df = pd.read_table('MouseAgingClocks/ClockData/Thompson2018-ElasticNet_aging_clock.txt', skiprows=1)

intercept = df['Coefficient'].iloc[0]

df = df[1:]

df['feature'] = df['Chromosome'].astype(str) + ':' + df['Coordinate'].astype(int).astype(str)
df['coefficient'] = df['Coefficient']

In [4]:
features = df['feature'].tolist()

weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([intercept])

In [5]:
model = pya.models.LinearModel(len(features))

model.linear.weight.data = weights
model.linear.bias.data = intercept

model

LinearModel(
  (linear): Linear(in_features=582, out_features=1, bias=True)
)

In [6]:
weights_dict = {
    'preprocessing': None, 
    'preprocessing_helper': None,
    'postprocessing': None,
    'postprocessing_helper': None,
    'features': features,
    'weight_dict': model.state_dict(),
    'model_class': 'LinearModel',
}

metadata_dict = {
    'species': 'Mus musculus',
    'data_type': 'methylation',
    'year': 2018,
    'implementation_approved_by_author(s)': '✅',
    'preprocessing': weights_dict['preprocessing'], 
    'postprocessing': weights_dict['postprocessing'], 
    'citation': "Thompson, Michael J., et al. \"A multi-tissue full lifespan epigenetic clock for mice.\" Aging (Albany NY) 10.10 (2018): 2832.",
    'doi': "https://doi.org/10.18632/aging.101590",
    "notes": None,
}

In [7]:
torch.save(weights_dict, '../weights/thompsonmultitissue.pt')
torch.save(metadata_dict, '../metadata/thompsonmultitissue.pt')

In [8]:
os.system("rm -r MouseAgingClocks")

0